# Gym Pose Aid Colab

In [ ]:
!pip -q install tensorflow==2.19.0 tensorflow-hub==0.16.1 opencv-python numpy imageio imageio-ffmpeg dtaidistance matplotlib>=3.9import os, sysif 'google.colab' in sys.modules:
    os.environ['TFHUB_CACHE_DIR'] = '/content/tfhub_cache'

In [ ]:
import sys, os, numpy as npsys.path.append('../src')import tensorflow as tf, tensorflow_hub as hubfrom movenet_infer import run_movenet, draw_skeletonfrom io_utils import read_video, save_video, save_gif, download_samplefrom gestures import SimpleGestureDetectorfrom dtw_utils import dtw_distanceMODEL_URL = 'https://tfhub.dev/google/movenet/singlepose/lightning/4'movenet = hub.load(MODEL_URL)print('TF', tf.__version__)print('Keypoints shape:', run_movenet(movenet, np.zeros((192,192,3), dtype=np.uint8)).shape)

In [ ]:
video_path = Noneif 'google.colab' in sys.modules:    from google.colab import files    uploaded = files.upload()    if uploaded: video_path = list(uploaded.keys())[0]if not video_path:    video_path = download_sample('out')print('Video path:', video_path)fps, frames = read_video(video_path)detector = SimpleGestureDetector()out_frames = []kseq = []events = []for i, frame in enumerate(frames):    kpts = run_movenet(movenet, frame)[0,0]    kseq.append(kpts)    out_frames.append(draw_skeleton(frame, kpts))    ev = detector.update(kpts)    for name, flag in ev.items():        if flag:            events.append({'t_sec': i/fps, 'event': name})save_video('out/pose_overlay.mp4', out_frames, fps)save_gif('out/pose_overlay.gif', out_frames, fps)import csvif events:    with open('out/events.csv','w',newline='') as f:        writer = csv.DictWriter(f, fieldnames=['t_sec','event'])        writer.writeheader(); writer.writerows(events)print('Saved outputs to out/')

In [ ]:
tmpl = kseq[:30]cmp = kseq[30:60] if len(kseq) >= 60 else kseq[:30]dist = dtw_distance(tmpl, cmp)print('DTW distance:', dist)

In [ ]:
import matplotlib.pyplot as pltplt.imshow(frames[0]); plt.axis('off')plt.figure(); plt.imshow(out_frames[0]); plt.axis('off')

In [ ]:
events[:5]

In [ ]:
from IPython.display import HTMLHTML('<video controls src="out/pose_overlay.mp4"></video>')